In [1]:

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import os
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI




In [2]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [10]:
# load document
loader = PyPDFLoader("../../example_paper1.pdf")
documents = loader.load()
print(len(documents))


33


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
text = text_splitter.split_documents(documents)
print(len(text))

In [ ]:
embeddings = OpenAIEmbeddings()
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents=text, 
                           embedding=embeddings)
print(db._collection.count())

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
COHERE_API_KEY = os.environ["COHERE_API_KEY"] 

# from typing import ForwardRef
# from pydantic import BaseModel
# from langchain.retrievers.document_compressors import CohereRerank

# class CustomCohereRerank(CohereRerank):
#     class Config(BaseModel.Config):
#         arbitrary_types_allowed = True
# co = Client()
# CustomCohereRerank.update_forward_refs()

# compressor = CustomCohereRerank(client=co)
# retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k":4})
# compressor = CohereRerank(user_agent="my-app")
# compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)


llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff", 
    return_source_documents=True
)

question = "what are the key methods for building trustworthy AI?"
# result = qa_chain({"query": question})
# print(result)

docs = retriever.get_relevant_documents(question)
# compressed_docs = compression_retriever.get_relevant_documents(question)



In [3]:
from typing import Any
import prompts


class Langchain_app:

    def __init__(self):
        self.docs = None
        self.db = None

    def load_data(self, filepath):
        loader = PyPDFLoader(filepath)
        self.docs = loader.load()
        return self
    
    def indexing(self):

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
        text = text_splitter.split_documents(self.docs)
        embeddings = OpenAIEmbeddings()
        db = Chroma.from_documents(documents=text, 
                                embedding=embeddings)
        self.db = db
        return self
    
    def retrieving(self):
        
        retriever = self.db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k":4})
        self.retriever = retriever
        return self
    
    def responding(self, query):
        llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=self.retriever,
            chain_type="stuff", 
            return_source_documents=True
        )
        self.query_engine = qa_chain
        response = qa_chain(query)

        return response
    
    def summarise(self, filepath):

        self.load_data(filepath)
        self.indexing()
        self.retrieving()

        query = "What are the key findings disucssed? Please summarise them in bullet points."
        response = self.responding(query)
        return response

        

In [4]:
rag1 = Langchain_app()
response = rag1.summarise("../../example_paper1.pdf")

In [5]:
response

{'query': 'What are the key findings disucssed? Please summarise them in bullet points.',
 'result': '- The study explores the performance of tree-based models on tabular data.\n- The study raises questions about other inductive biases of tree-based models that contribute to their performance on tabular data.\n- The evaluation of tree-based models on small and large datasets, as well as datasets with missing data, is left as future work.\n- The study reveals that tree-based models easily yield good predictions on tabular data with less computational cost.\n- The study includes a figure showing the test accuracy of a Gradient Boosting Tree (GBT) for varying proportions of removed features, indicating the importance of features in the prediction accuracy.',
 'source_documents': [Document(page_content='other ways to break rotation invariance which might be less computationally costly than embeddings.\n6 Discussion and conclusion\nLimitation Our study leaves open questions for future work:

In [7]:
print(response['result'])

- The study explores the performance of tree-based models on tabular data.
- The study raises questions about other inductive biases of tree-based models that contribute to their performance on tabular data.
- The evaluation of tree-based models on small and large datasets, as well as datasets with missing data, is left as future work.
- The study reveals that tree-based models easily yield good predictions on tabular data with less computational cost.
- The study includes a figure showing the test accuracy of a Gradient Boosting Tree (GBT) for varying proportions of removed features, indicating the importance of features in the prediction accuracy.


In [11]:
from langchain.chains.summarize import load_summarize_chain
llm = ChatOpenAI(temperature=0)
chain = load_summarize_chain(llm, chain_type="map_reduce")
print(chain.run(documents))

The study conducted extensive benchmarks comparing tree-based models and deep learning models on tabular data. Tree-based models, such as XGBoost and Random Forests, outperformed deep learning models on medium-sized datasets. The study investigated the inductive biases of tree-based models and neural networks, highlighting challenges in building tabular-specific neural networks. The authors provided a standard benchmark and raw data to stimulate research in this area. The study also identified limitations of deep learning models in learning irregular patterns and rotation invariance. The authors hope their findings will assist in building successful deep learning models for tabular data. The study included a comprehensive benchmark with 45 datasets and addressed the lack of a standard benchmark in the field. The authors also conducted experiments on large-scale datasets and found that increasing the train set size reduced the performance gap between neural networks and tree-based model

In [ ]:
from trulens_eval import Tru
from utils import get_prebuilt_trulens_recorder_langchain
tru = Tru()
tru.reset_database()

# eval_questions = ["who is the author of this article?", "what technique does this paper use?"]

# def run_evals(eval_questions, tru_recorder, query_engine):
#     for question in eval_questions:
#         with tru_recorder as recording:
#             response = query_engine(question)


# tru_recorder_langchain = get_prebuilt_trulens_recorder_langchain(
#     qa_chain,
#     app_id = "Langchain Query Engine"
# )

In [ ]:
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI,
    TruChain,
    Select
)
import numpy 
from trulens_eval.feedback import Groundedness
openai = OpenAI()

qa_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)
qs_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
    .on_input_output()
)

grounded = Groundedness(groundedness_provider=openai)

groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content)
        .on_output()
        .aggregate()
)

truchain = TruChain(
        rag1.query_engine,
        app_id='Chain1_ChatApplication',
        feedbacks=[qa_relevance, qs_relevance, groundedness]
        )

In [ ]:
Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content(rag1.query_engine)

In [ ]:
question = "what are the key pillars for trustworthy AI?"
with truchain as recording:
    rag1(question)

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0]["Groundedness"]

In [ ]:
tru.get_leaderboard(app_ids=["Chain1_ChatApplication"])

In [ ]:
from trulens_eval import Tru
tru.run_dashboard()

In [ ]:
run_evals(eval_questions, tru_recorder_langchain, qa_chain)

In [ ]:
Tru.run_dashboard()